# MONTE CARLO SEQUENCIAL

## IMPORTAÇÃO DE BIBLIOTECAS

In [1]:
from pyomo.environ import *
import pandas as pd
import numpy as np
from data_processing import load_all_data
from calcula_fpo import calcular_fpo

0.0


## LEITURA DE DADOS

In [2]:
data = load_all_data()
D_GEN = data['D_GEN']
D_LIN = data['D_LIN']
D_LOAD = data['D_LOAD']

In [3]:
#Deixando os dados em horas e nos formatos conhecidos
#Dados de Linha
D_GEN['Lambda'] = D_GEN['Falhas/ano'] / 8760 # 8760 horas em um ano
D_GEN['MTTF'] = 1/D_GEN['Lambda'] #Tempo médio entre falhas
D_GEN['mu'] = 1/D_GEN['Tempo de reparo(h)'] #Taxa de reparo/hora
D_GEN['A'] = D_GEN['mu']/(D_GEN['mu'] + D_GEN['Lambda']) #Disponibilidade
D_GEN['U'] = 1 - D_GEN['A'] #Indisponibilidade

#Dados de Linha
D_LIN['Lambda'] = D_LIN['Falhas/(ano.milha)'] * D_LIN['Comprimento'] / 8760 # 8760 horas em um ano
D_LIN['MTTF'] = 1 / D_LIN['Lambda'] #Tempo médio entre falhas
D_LIN['mu'] = 1 / D_LIN['Tempo de Reparo'] #Taxa de reparo/hora
D_LIN['A'] = D_LIN['mu'] / (D_LIN['mu'] + D_LIN['Lambda']) #Disponibilidade
D_LIN['U'] = 1 - D_LIN['A'] #Indisponibilidade

In [4]:
Ng = len(D_GEN)
geradores = np.arange(1,Ng+1)
Nl = len(D_LIN)
linhas = np.arange(1,Nl+1)
Nc = len(D_LOAD)
cargas = np.arange(1,Nc+1)

## FUNÇÕES AUXILIARES

In [5]:
# Função de cálculo do tempo de residência (TR)
def sortear_tempo_residencia(taxa):
    return float(np.random.exponential(1 / taxa))

def calcular_estatisticas(resultados):
    if len(resultados) > 1:
        # Calcular a média
        media = sum(resultados) / len(resultados)
        
        # Calcular a variância (usando a fórmula da variância amostral)
        variancia = sum((x - media) ** 2 for x in resultados) / (len(resultados) - 1)  # ddof=1 para variância amostral
        
        # Calcular a variância dividida pelo tamanho da lista
        variancia_media = variancia / len(resultados)
        
        # Calcular o Beta
        beta = (variancia_media ** 0.5) / media 
        return media, variancia, beta
    else:
        return 10, 10, 10


## ALGORITMO PRINCIPAL MCS

In [6]:
#Parâmetros principais:
tol = 0.01 #Tolerância para  o critério de parada
NY_MAX = 100 #Máximo número de anos para simular
NY = 0 #Contador de anos
convergiu = False
beta = 10

#Vetores que armazenam os resultados de cada ano simulado
resultados_LOLP = []
resultados_LOLE = []
resultados_EPNS = []
resultados_EENS = []
resultados_LOLF = []
resultados_LOLD = []

In [ ]:
while not convergiu:
    NY += 1
    # Reinicializar estados dos componentes no início de cada ano
    estados_geradores = np.ones(Ng, dtype=int)
    estados_linhas = np.ones(Nl, dtype=int)

    # Sorteio dos tempos de residência para todos os componentes
    TG = [sortear_tempo_residencia(D_GEN.iloc[k]['Lambda']) for k in range(Ng)]
    TL = [sortear_tempo_residencia(D_LIN.iloc[k]['Lambda']) for k in range(Nl)]

    # Passo e: Inicialização de variáveis no início do ano
    TSIS = 0
    horas_com_corte = 0
    energia_nao_suprida = 0
    eventos_corte = 0
    F_LOLE = 0
    F_EENS = 0
    F_LOLF = 0

    while TSIS < 8760:
        # Passo f: Identificar o componente com menor TG ou TL
        menor_TG = min(TG)
        menor_TL = min(TL)

        if menor_TG < menor_TL:
            i = np.argmin(TG)
            TGi = menor_TG
            componente = 'gerador'
            if estados_geradores[i] == 1:
                estados_geradores[i] = 0
            else:
                estados_geradores[i] = 1
        else:
            i = np.argmin(TL)
            TGi = menor_TL
            componente = 'linha'
            if estados_linhas[i] == 1:
                estados_linhas[i] = 0
            else:
                estados_linhas[i] = 1

        # Passo g: Ajustar TG ou TL se maior que 8760 (1 ano)
        if TGi > 8760:
            TGi = 8760

        # Passo h: Calcular TDUR e atualizar TSIS
        TDUR = TGi - TSIS
        TSIS = TGi

        z_ger = estados_geradores
        z_lin = estados_linhas
        corte_carga = calcular_fpo(z_ger, z_lin, False) #Calcula quantos MW foram cortados com a configuração de componentes atual

        if corte_carga > 0:
            horas_com_corte += TDUR
            energia_nao_suprida += corte_carga * TDUR
            eventos_corte += 1

        #Passo j (vem antes do passo i): Sorteio de novo TR para i e atualização de TG
        if componente == 'gerador':
            if estados_geradores[i] == 1:
                TR = sortear_tempo_residencia(D_GEN.iloc[i]['Lambda'])
            else:
                TR = sortear_tempo_residencia(D_GEN.iloc[i]['mu'])
            TG[i] = TGi + TR
        else:
            if estados_linhas[i] == 1:
                TR = sortear_tempo_residencia(D_LIN.iloc[i]['Lambda'])
            else:
                TR = sortear_tempo_residencia(D_LIN.iloc[i]['mu'])
            TL[i] = TGi + TR 

    #Cálculo dos índices anuais
    LOLP = horas_com_corte/8760
    LOLE = horas_com_corte
    EPNS = energia_nao_suprida/8760
    EENS = energia_nao_suprida
    LOLF = eventos_corte
    LOLD = LOLE / eventos_corte if eventos_corte > 0 else 0

    # Armazenar resultados anuais em vetores
    resultados_LOLP.append(LOLP)
    resultados_LOLE.append(LOLE)
    resultados_EPNS.append(EPNS)
    resultados_EENS.append(EENS)
    resultados_LOLF.append(LOLF)
    resultados_LOLD.append(LOLD)
    
    if NY > 2:
        # Cálculo dos índices estimados e seus coeficientes de convergência
        media_LOLP, var_LOLP, beta_LOLP = calcular_estatisticas(resultados_LOLP)
        media_LOLE, var_LOLE, beta_LOLE = calcular_estatisticas(resultados_LOLE)
        media_EPNS, var_EPNS, beta_EPNS = calcular_estatisticas(resultados_EPNS)
        media_EENS, var_EENS, beta_EENS = calcular_estatisticas(resultados_EENS)
        media_LOLF, var_LOLF, beta_LOLF = calcular_estatisticas(resultados_LOLF)
        media_LOLD, var_LOLD, beta_LOLD = calcular_estatisticas(resultados_LOLD)
        # Armazenar os betas em um vetor e seleciona o maior:
        beta = max([beta_LOLP, beta_EPNS])
    if beta < tol or NY > NY_MAX:
        convergiu = True

In [17]:
#Cálculo do desvio padrão da média dos índices de confiabilidade:
std_LOLP = np.std(resultados_LOLP)
stdd_LOLE = np.std(resultados_LOLE)
std_LOLF = np.std(resultados_LOLF)
std_EENS = np.std(resultados_EENS)
std_EPNS = np.std(resultados_EPNS)
std_LOLD = np.std(resultados_LOLD)

#Cálculo do erro padrão da média dos índices de confiabilidade:
erro_padrao_LOLP = std_LOLP / np.sqrt(NY)
erro_padrao_LOLE = stdd_LOLE / np.sqrt(NY)
erro_padrao_LOLF = std_LOLF / np.sqrt(NY)
erro_padrao_EENS = std_EENS / np.sqrt(NY)
erro_padrao_EPNS = std_EPNS / np.sqrt(NY)
erro_padrao_LOLD = std_LOLD / np.sqrt(NY)

# Print dos Resultados:
print(f"Expected Energy Not Supplied (EENS): ({media_EENS/1000:.2f} ± {erro_padrao_EENS/1000:.3f}) GWh/ano")
print(f"Expected Power Not Supplied (EPNS): ({media_EPNS:.2f} ± {erro_padrao_EPNS:.3f}) MW")
print(f"Loss of Load Duration (LOLD): ({media_LOLD:.2f} ± {erro_padrao_LOLD:.3f}) horas/eventos de corte")
print(f"Loss of Load Expectation (LOLE): ({media_LOLE:.2f} ± {erro_padrao_LOLE:.3f}) horas/ano")
print(f"Loss of Load Probability (LOLP): ({media_LOLP*100:.2f} ± {erro_padrao_LOLP*100:.3f}) %")
print(f"Loss of Load Frequency (LOLF): ({media_LOLF:.2f} ± {erro_padrao_LOLF:.3f}) eventos de corte/ano")

Expected Energy Not Supplied (EENS): (161.19 ± 5.393) GWh/ano
Expected Power Not Supplied (EPNS): (18.40 ± 0.616) MW
Loss of Load Duration (LOLD): (28.52 ± 0.551) horas/eventos de corte
Loss of Load Expectation (LOLE): (1584.40 ± 35.185) horas/ano
Loss of Load Probability (LOLP): (18.09 ± 0.402) %
Loss of Load Frequency (LOLF): (56.36 ± 1.150) eventos de corte/ano
